# Rešavanje sistema linearnih jednačina na kvantnom računaru

Postoji nekoliko algoritama za rešavanje sistema linearnih jednačina na kvantnom računaru:
* algoritam HHL, nazvan prema autorima Harou (eng. *Harrow*), Hasidimu (eng. *Hassidim*) i Lojdu (eng. *Lloyd*)
* algoritam AQC(p)
* algoritam AQC(exp)
* algoritam zasnovan na kvantnim algoritmima aproksimativne optimizacije (eng. *Quantum Approximate Optimization Algorithms*, skraćeno QAOA).

Poslednja tri algoritma koje smo naveli su definisali autori Dōng Àn i njegov mentor Lín Lín i oni se zasnivaju na adijabatskom kvantnom izračunavanju. Sada ćemo opisati zajedničke karakteristike svih navedenih algoritama, pa ćemo razmatrati svaki zasebno.

Neka je data invertibilna matrica $A$ dimenzija $N \times N$ i vektor $b$ dužine $N$. Sistem linearnih jednačina koji rešavamo je:
\begin{align*}
    Ax = b
\end{align*}

Kada je matrica ermitska (brzo ćemo videti kako se svodi na taj slučaj), konstanta $\kappa = \max\{|\lambda|\} / \min\{\|\lambda|\}$ je dobro definisana i pritom važi $\kappa \geq 1$. Konstanta $\kappa$ je kondicioni broj ermitske matrice $A$ i on nam govori koliko je $A$ blizu neke neinvertibilne matrice - što je veći kondicioni broj, to je matrica $A$ bliža neinvertibilnoj matrici. Ova konstantna je jako bitna, jer određuje stabilnost rešavanja sistema linearnih jednačina $Ax = b$.

Svi algoritmi koje smo naveli su numerički i, osim kondicionog broja $\kappa$, bitnu ulogu igra i greška $\epsilon$ u vremenskoj složenosti algoritama.

Dalje, da bi se algoritmi sproveli na kvantnom računaru, dimenzije matrice $A$ se usklađuju tako da je $N = 2^n$ za neko $n \in \mathbb{N}$. Ako to nije slučaj, matricu $A$ možemo dopuniti tako što ćemo dodavati vrste i kolone koje na glavnoj dijagonali imaju $1$ dok ne postane oblika $2^n \times 2^n$, i dodati na kraj vektora $b$ nule dok ne postane dužine $2^n$ za neko $n$.

Paulijeve matrice označavamo sa $X$, $Y$ i $Z$. Definišimo operatore $\sigma_{+} = \frac{1}{2} (X + i Y)$ i $\sigma_{-} = \frac{1}{2} (X - i Y)$.

Osnovni koraci svakog od ovih algoritama su:
1. normalizacija vektora $b$, odnosno $b = c b'$, gde je $b'$ jedinični vektor;
2. nalaženje vektora $x''$ takav da je $x''$ proporcionalan rešenju $x$ sistema $A x' = b'$ - ovde se u sva četiri algoritma matrica $A$ svodi na ermitsku matricu tako što se definiše matrica $\begin{pmatrix} 0 & A \\ A^{\dagger} & 0 \end{pmatrix}$ izrazom $\sigma_{+} \otimes A + \sigma_{-} \otimes A^{\dagger}$, a vektor $b'$ zamenjujemo vektorom $\ket{1} \otimes \ket{b'}$.

Ovakvo rešavanje sistema linearnih jednačina nam ne daje zapravo vektor $x$ takav da je $Ax = b$, već nam daje vektor $x''$ koji je proporcionalan rešenju $A x' = b'$. Zato se u literaturi ovo preciznije naziva *kvantno rešavanje sistema linearnih jednačina*. U nekim optimizacionim problema, kao što je na primer *portfolio optimization*, ovo ima primenu. Dalje ćemo prikazati kako ovo radi.

Prva glavna pretpostavka sva četiri algoritma je da nam je dato kvantno kolo $U_b$ takvo da je $U_b \ket{0} = \ket{b'}$. To kvantno kolo priprema vektor $b'$. Konstrukcija ovakvog kvantnog kola nije trivijalna.

Druga glavna pretpostavka je da je matrica $A$ retka, odnosno da ima mali broj ne-nula elemenata u sebi. Ta pretpostavka je ključna, jer inače algoritmi postaju jako neefikasni.

## Algoritmi autora Dōng Àn i Lín Lín

Ovde se već nalazimo u koraku $2$. Dakle, možemo pretpostaviti da je $A$ ermitska matrica i da je $b$ jedinični vektor. Rešavamo sistem jednačina $A x = b$. Osnovna ideja algoritma je da se definišu ermitski operatori $H_0$ i $H_1$, pa da se primeni operator vremenske evolucije, kojim se $H_0$ svodi na $H_1$. $H_0$ i $H_1$ su konstruisani tako da je sopstveni vektor $\ket{\psi_0}$ uz $0$ od $H_0$ poznata vrednost, a sopstveni vektor $\ket{\psi_1}$ uz $0$ od $H_1$ je takav da se iz njega može jednostavno izračunati $x''$. Korišćenjem adijabatske teoreme iz kvantne mehanike, autori pokazuju da će vektor $\ket{\psi_0}$ zaista evoluirati u vektor $\ket{\psi_1}$, čime se potvrđuje tačnost algoritma. Sva tri algoritma se zasnivaju na toj ideji, samo je drugačija realizacija.

Ono što je zajedničko svim algoritmima su operatori $H_0$ i $H_1$. Ovde autori posmatraju dva slučaja:
1. ako je $A$ pozitivno definitna matrica, onda je:
   \begin{align*}
       Q_b &= I_N - \ket{b}\bra{b}, \\
       H_0 &= X \otimes Q_b, \\
       H_1 &= \sigma_{+} \otimes (A Q_b) + \sigma_{-} \otimes (Q_b A).
   \end{align*}

   Ovde se može primetiti da su sopstveni vektori uz $0$ od operatora $H_0$ $\ket{1} \otimes \ket{b}$ i $\ket{0} \otimes \ket{b}$, a od operatora $H_1$ su $\ket{1} \otimes \ket{x''}$ i $\ket{0} \otimes \ket{b}$. Ovde je sada ideja da se adijabatskim računanjem stanje $\ket{1} \otimes \ket{b}$ prevede u stanje $\ket{1} \otimes \ket{x''}$
2. ako $A$ nije pozitivno definitna matrica, onda je:
   \begin{align*}
       Q_{+, b} &= I_{2N} - \ket{+}\bra{+} \otimes \ket{b}\bra{b}, \\
       H_0 &= \sigma_{+} \otimes ((Z \otimes I_N) Q_{+, b}) + \sigma_{-} \otimes (Q_{+, b} (Z \otimes I_N)), \\
       H_1 &= \sigma_{+} \otimes ((X \otimes A) Q_{+, b}) + \sigma_{-} \otimes (Q_{+, b} (X \otimes A)).
   \end{align*}

   Primetimo da su sopstveni vektori uz $0$ od operatora $H_0$ $\ket{0}\ket{-}\ket{b}$ i $\ket{1}\ket{+}\ket{b}$, a od $H_1$ $\ket{0}\ket{+}\ket{x''}$ i $\ket{1}\ket{+}\ket{b}$. Sada je ideja da se adijabatskim računanjem stanje $\ket{0}\ket{-}\ket{b}$ prevede u $\ket{0}\ket{+}\ket{x''}$.

Sada već imamo $H_0$, $H_1$ i $\ket{\psi_{0}}$. Sada je potrebno opisati kako se vremenskom evolucijom $\ket{\psi_0}$ svodi na $\ket{\psi_1}$. Za to postoje dva pristupa.

### AQC(p)

Protok vremena se može opisati funkcijom:
\begin{align*}
    f_p(s) = \frac{\kappa}{\kappa - 1} (1 - (1 + s(\kappa^{p - 1} - 1))^{\frac{1}{1 - p}}),
\end{align*} za $1 < p \leq 2, $odnosno $H_0 = H(f_p(0))$ i $H_1 = H(f_p(1))$, jer $f_p(0) = 0$ i $f_p(1) = 1$.

U ovom slučaju, biramo konstantu $T = O(\kappa / \epsilon)$ za $1 < p < 2$ i $T = O(\kappa \log \kappa / \epsilon)$ za $p = 1, 2$.

### AQC(exp)

U ovom slučaju, vreme se opisuje funkcijom:
\begin{align*}
    f(s) = c_e^{-1} \int_{0}^{s} \exp \left(-\frac{1}{x (1 - x)}\right) dx,
\end{align*} gde je $c_e = \int_{0}^{1} \exp \left(-\frac{1}{x (1 - x)}\right) dx$ konstanta normalizacije tako da $f(1) = 1$.

U ovom slučaju, bira se konstanta $T = O(\kappa \log^2 \kappa \log^4 (\log \kappa / \epsilon))$.

### Vremenska evolucija

Nakon što smo izabrali $T$, sada koristimo Suzuki-Troterovu formulu za simulaciju vremenske evolucije sistema. U oba slučaja se koristi isti operator:
\begin{align*}
    \prod_{m = 1}^{M} exp(-i T h (1 - f(s_m)) H_0) exp(-i T h f(s_m) H_1),
\end{align*} gde je $h = 1 / M$, a $s_m = m h$ i $M = O(\text{polylog}(N) T^2 / \epsilon)$.

### QAOA

Autori su koristili Hamiltonijan $H_0$ kao mikser, a Hamiltonijan $H_1$ kao Hamiltonijan cene. Oni tvrde da se dobra aproksimacija može naći optimizacijom parametara $\beta_i$ i $\gamma_i$, za $1 \leq i \leq k$, gde je $k$ broj slojeva u pretpostavljenom rešenju:
\begin{align*}
    e^{- i \gamma_k H_1} e^{-i \beta_k H_0} \ldots e^{-i \gamma_1 H_1} e^{-i \beta_1 H_0} \ket{\psi_0}.
\end{align*}

Parametri se optimizuju tako da je vrednost $\bra{\psi(\gamma, \beta)} H_1^2 \ket{\psi(\gamma, \beta)}$ minimalna. Optimizacija se izvršava sve dok je trenutna vrednost minimuma veća ili jednaka od $\epsilon^2 / \kappa^2$. Za optimizaciju se mogu koristiti klasični algoritmi, kao što su **COBYLA**, gradijentni spust, algoritam Neldera i Mida (eng. Nelder-Mead).

Rešenje koje se dobije ovim putem predstavlja vektor $\ket{x''}$.

### Određivanje kondicione vrednosti

Bitna stavka prva dva algoritma je kondiciona vrednost matrice $A$. Trenutno ne postoje efikasni algoritmi na kvantnom računaru za izračunavanje ove vrednosti. Za izračunavanje $\max\{|\lambda|\}$ je moguće koristiti algoritam VQE, ali izračunavanje $\min\{|\lambda|\}$ je teško. Postoje gornja ograničenja na kondicioni broj koja zahtevaju da se izračunaju determinanta i Frobeniusova norma matrice $A$. Izračunavanje determinante isto nije lak zadatak - na klasičnom računaru zahteva $O(n^3)$ koraka. 